Url: https://tbrain.trendmicro.com.tw/Competitions/Details/2

In [30]:
#import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import Row
from pyspark.sql.functions import col, udf, lag, rank, lit
from pyspark.sql.window import Window

In [31]:
global Path
if sc.master[0:5]=="local":
    Path = "file:/c:/D Drive/work/bigData/pySpark/TBrain_Round2_DataSet_20180427"
    #Path = "file:/Users/yungchuanlee/Documents/learn/AI競賽/ETF預測/TBrain_Round2_DataSet_20180427"
    #Path = "file:/home/hduser/app/bigdata/competition/etf/TBrain_Round2_DataSet_20180427"
else:
    Path = "hdfs://master:9000/user/hduser"

In [32]:
sc.master

'local[*]'

In [33]:
float("    46.57")+float("     29,020".replace(",",""))

29066.57

In [34]:
#define alias of columns
col_alias_etf= {"代碼":"etf_id", "日期": "etf_date", "中文簡稱": "etf_name", "開盤價(元)":"etf_open", 
            "最高價(元)":"etf_high", "最低價(元)":"etf_low", "收盤價(元)":"etf_close", "成交張數(張)":"etf_count"}
col_alias_stock= {"代碼":"stock_id", "日期": "stock_date", "中文簡稱": "stock_name", "開盤價(元)":"stock_open", 
            "最高價(元)":"stock_high", "最低價(元)":"stock_low", "收盤價(元)":"stock_close", "成交張數(張)":"stock_count"}

In [35]:
#udf
def to_double(str_val):
    return float(str_val.replace(",",""))
to_double=udf(to_double)

In [36]:
#def function to read data (因檔案格式都相同)
def read_data(file_name, col_alias):
    str_cols = ["代碼","日期", "中文簡稱"]
    raw_data = spark.read.option("encoding", "Big5").csv(Path + "/" + file_name, header=True, sep=",")
    print("Total " + file_name + " count: " + str(raw_data.count()))
    #rename cols and correct type 
    num_cols = [col_name for col_name in raw_data.columns if col_name not in str_cols]
    final_data=raw_data.select( [col(str_col_name).alias(col_alias[str_col_name]) for str_col_name in str_cols] + 
                                  [to_double(col(num_col_name)).cast("double").alias(col_alias[num_col_name]) for num_col_name in num_cols] )
    final_data.printSchema()
    final_data.show(5)
    return final_data

In [37]:
print("starting import tetfp.csv(台灣18檔ETF股價資料)...")
tetfp_dt=read_data("tetfp.csv", col_alias_etf)

starting import tetfp.csv(台灣18檔ETF股價資料)...
Total tetfp.csv count: 6515
root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- etf_name: string (nullable = true)
 |-- etf_open: double (nullable = true)
 |-- etf_high: double (nullable = true)
 |-- etf_low: double (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- etf_count: double (nullable = true)

+-------+--------+----------------+--------+--------+-------+---------+---------+
| etf_id|etf_date|        etf_name|etf_open|etf_high|etf_low|etf_close|etf_count|
+-------+--------+----------------+--------+--------+-------+---------+---------+
|0050   |20130102|元大台灣50          |    54.0|   54.65|   53.9|     54.4|  16487.0|
|0050   |20130103|元大台灣50          |    54.9|   55.05|  54.65|    54.85|  29020.0|
|0050   |20130104|元大台灣50          |   54.85|   54.85|   54.4|     54.5|   9837.0|
|0050   |20130107|元大台灣50          |   54.55|   54.55|   53.9|    54.25|   8910.0|
|0050   |20130108|元大台灣50      

In [38]:
#EDA
#range of date
tetfp_dt.describe('etf_date').show()

+-------+--------------------+
|summary|            etf_date|
+-------+--------------------+
|  count|                6515|
|   mean|2.0152356018419035E7|
| stddev|  15378.548560182882|
|    min|            20130102|
|    max|            20180427|
+-------+--------------------+



In [39]:
# print("starting import taetfp.csv(台灣18檔ETF調整後股價資料)...")
# taetfp_dt=read_data("taetfp.csv", col_alias_etf)

In [40]:
# print("starting import tsharep.csv(台灣個股股價資料)...")
# tsharep_dt=read_data("tsharep.csv", col_alias_stock)

In [41]:
# print("starting import tasharep.csv(台灣個股調整後股價資料)...")
# tasharep_dt=read_data("tasharep.csv", col_alias_stock)

In [58]:
import sys
from pyspark.sql.functions import lag, col, avg,collect_list, lit
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, DoubleType, IntegerType
#declare previous row windows
wsSpec_etf = Window.partitionBy('etf_id').orderBy('etf_date') #time window for normal case
wsSpec_etf_close_price_raw = Window.partitionBy('etf_id').orderBy('row_idx').rangeBetween(-sys.maxsize, -1)
wsSpec_etf_dif_raw = Window.partitionBy('etf_id').orderBy('row_idx').rangeBetween(-sys.maxsize, 0)
def avg_list(p_list):
    #計算數字list的平均值
    return sum(p_list)/len(p_list)
#計算EMA的udf
def calculate_ema_native(close_p_list, window_len):
    #透過歷史收盤價計算
    if len(close_p_list) < window_len:
        return None
    elif len(close_p_list) == window_len:
        #if len of list = win_len then return avg, 
        return avg_list(close_p_list)
    else:
        #else EMA[t] =(EMA[t-1]*(win_len-1)+close[t]*2)/(win_len+1)
        ema = avg_list(close_p_list[:window_len])
        for price in close_p_list[window_len:]:
            ema = (ema*(window_len-1)+price*2)/(window_len+1)
        return ema
calculate_ema=udf(calculate_ema_native, DoubleType())
#計算BIAS的udf
def calculate_bias(close_p_list):
    #計算前日收盤價與N日均線之差比: (close price - MA)/MA   ,Paper 建議用20日MA
    #因要預測今日的收盤價，故計算前日收盤價與前20日均線
    if len(close_p_list) < 21:
        return None
    else:
        list_len = len(close_p_list)
        p_close = close_p_list[-1]
        cal_list = close_p_list[list_len-21: list_len-1]
        return p_close - avg_list(cal_list)
calculate_bias=udf(calculate_bias, DoubleType())

def get_min_max_last(p_list):
    #找出list中最大最小和最後一個值, 回傳(min, max, last)
    return (min(p_list), max(p_list), p_list[-1])
def calculate_raw_rsv(p_list):
    #RSV = (收盤價-9日低值)/(9日高值-9日低值)
    p_min, p_max, p_last = get_min_max_last(p_list)
    rsv = (p_last - p_min)/(p_max - p_min)
    return rsv
def calculate_rsv(p_9_list, k_prev, d_prev):
    #計算加權後的RSV，p_9_list=>9日收盤價
    rrsv = calculate_raw_rsv(p_9_list)
    k_curr = (1/3)*rrsv + (2/3)*k_prev
    d_curr = (1/3)*k_curr + (2/3)*d_prev
    return [k_curr, d_curr]
#計算隨機指標（Stochastic Oscillator，KD），原名%K&%D
def calculate_KD(close_p_list):
    win_len = 9 #看過去 9 日值
    #RSV = (收盤價-9日低值)/(9日高值-9日低值)
    #K_curr = 1/3*RSV + 2/3*K_prev
    #D_curr = 1/3*K_curr + 2/3*D_prev
    if len(close_p_list) < win_len:
        return None
    elif len(close_p_list) == win_len:
        #無前日K, D時，以0.5帶入
        return calculate_rsv(close_p_list, 0.5, 0.5)
    else:
        kds = calculate_rsv(close_p_list[0:9], 0.5, 0.5)
        for idx in range(1, (len(close_p_list)+1-9)):
            p_9_list = close_p_list[idx: idx+9]
            kds = calculate_rsv(p_9_list, kds[0], kds[1])
        return kds
calculate_KD=udf(calculate_KD, ArrayType(DoubleType()))

#計算差離值DIF = 12日EMA - 26日EMA
def calculate_DIF_native(close_p_list):
    if len(close_p_list) < 26:
        return None
    else:
        ema12 = calculate_ema_native(close_p_list, 12)
        ema26 = calculate_ema_native(close_p_list, 26)
        return ema12 - ema26
calculate_DIF=udf(calculate_DIF_native, DoubleType())

#計算MACD=(前一日MACD × (9 - 1) + 今日DIF × 2) ÷ (9 + 1)
def calculate_MACD(dif_list, dif_curr):
    win_len = 9
    if len(dif_list) < win_len:
        return None
    elif len(dif_list) == win_len:
        #if len of list = win_len then return avg, 
        return avg_list(dif_list)
    else:
        #MACD=(前一日MACD × (9 - 1) + 今日DIF × 2) ÷ (9 + 1)
        macd = avg_list(dif_list[:win_len])
        for price in dif_list[win_len:]:
            macd = (macd*(win_len-1)+dif_curr*2)/(win_len+1)
        return macd
calculate_MACD=udf(calculate_MACD, DoubleType())

#計算相對強弱指數(RSI)
def calculate_RSI(close_p_list):
    win_len = 9
    if len(close_p_list) < (win_len + 1):
        return None
    else:
        cur_list = close_p_list[1:]
        prv_list = close_p_list[0:-1]
        p_dif_list = list(map(lambda x,y : x - y, cur_list, prv_list)) #dif list
        u_list = []
        d_list = []
        for dif in p_dif_list:
            if dif == 0:
                #若兩天價格相同，則U及D皆等於零
                u_list.append(0)
                d_list.append(0)
            elif dif > 0:
                #在價格上升的日子, U = diff, D = 0
                u_list.append(dif)
                d_list.append(0)
            else:
                #在價格下跌的日子, U = 0, D = abs(diff)
                u_list.append(0)
                d_list.append(abs(dif))
        #RSI = ema(u,9)/(ema(u,9)+ema(d,9))
        ema_u = calculate_ema_native(u_list, win_len)
        ema_d = calculate_ema_native(d_list, win_len)
        return ema_u/(ema_u + ema_d)
calculate_RSI=udf(calculate_RSI, DoubleType())

#計算威廉指標（Williams %R）
def calculate_WR(close_p_list):
    win_len = 9
    if len(close_p_list) < win_len:
        return None
    else:
        p_list = close_p_list[len(close_p_list) - win_len :]
        return 1.0 - calculate_raw_rsv(p_list)
calculate_WR=udf(calculate_WR, DoubleType())


In [59]:
#calculate ema [5,10,20] #cannot remove row_idx, row_idx for next window usage
tetfp_dt2=tetfp_dt.withColumn("row_idx", rank().over(wsSpec_etf)) \
    .withColumn("close_price_raw", collect_list(col('etf_close')).over(wsSpec_etf_close_price_raw)) \
    .withColumn("EMA5", calculate_ema(col("close_price_raw"), lit(5))) \
    .withColumn("EMA10", calculate_ema(col("close_price_raw"), lit(10))) \
    .withColumn("EMA20", calculate_ema(col("close_price_raw"), lit(20))) \
    .withColumn("BIAS", calculate_bias(col("close_price_raw"))) \
    .withColumn("KD", calculate_KD(col("close_price_raw"))) \
    .withColumn("K", col("KD")[0]).withColumn("D", col("KD")[1]) \
    .withColumn("DIF", calculate_DIF(col("close_price_raw"))) \
    .withColumn("dif_list", collect_list(col('DIF')).over(wsSpec_etf_dif_raw)) \
    .withColumn("MACD", calculate_MACD(col("dif_list"), col("DIF"))) \
    .withColumn("RSI", calculate_RSI(col("close_price_raw")))\
    .withColumn("WR", calculate_WR(col("close_price_raw")))

tetfp_dt2.cache()
tetfp_dt2.printSchema()

root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- etf_name: string (nullable = true)
 |-- etf_open: double (nullable = true)
 |-- etf_high: double (nullable = true)
 |-- etf_low: double (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- etf_count: double (nullable = true)
 |-- row_idx: integer (nullable = true)
 |-- close_price_raw: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- EMA5: double (nullable = true)
 |-- EMA10: double (nullable = true)
 |-- EMA20: double (nullable = true)
 |-- BIAS: double (nullable = true)
 |-- KD: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- K: double (nullable = true)
 |-- D: double (nullable = true)
 |-- DIF: double (nullable = true)
 |-- dif_list: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- MACD: double (nullable = true)
 |-- RSI: double (nullable = true)
 |-- WR: double (nullable = true)



In [44]:
# tetfp_dt2.filter("etf_id='0050   '").select("row_idx", "etf_close", "MACD", "RSI", "WR") \
#             .show(45)
#          .toPandas().to_csv(Path.replace("file:","") + "/taetfp_BIAS.csv",index=False)

In [45]:
#匯出成pandas
# etf_pd_50 = tetfp_dt2.filter("etf_id='0050   '").select("row_idx", "etf_date" ,"etf_close", "EMA5", "EMA10", "EMA20", "BIAS") \
#                 .toPandas()

In [46]:
#圖形化顯示
#aetf_pd_50.set_index('etf_date') #set_index 後配合 loc select 出區段資料來看
# etf_pd_50_part = etf_pd_50.loc[1200:]
# etf_pd_50_part.etf_close.plot(x='row_idx', y='etf_close', style='b--', label="etf_close")
# etf_pd_50_part.EMA5.plot(x='row_idx', y='EMA5', label="EMA5", style='r-')
# etf_pd_50_part.EMA10.plot(x='row_idx', y='EMA10', label="EMA10", style='g-')
# etf_pd_50_part.EMA20.plot(x='row_idx', y='EMA20', label="EMA20", style='y-')
# plt.legend()
# plt.show()

In [47]:
#計算各欄位與收盤價之相關性
# corr_cols = ['EMA5','EMA10','EMA20','BIAS','K','D']
# for col in corr_cols:
#     print('corr between ', col , ' and etf_close: ', str(tetfp_dt2.corr(col, 'etf_close')))

In [60]:
from pyspark.ml.feature import MinMaxScaler, StandardScaler
from pyspark.ml.linalg import Vectors
tot_dt = tetfp_dt2.filter("MACD is not null") \
    .select("etf_id", "etf_date", "row_idx", "EMA5", "EMA10", "EMA20", "BIAS", "K", "D", "DIF", "MACD", "RSI", "WR", "etf_close") \
    .orderBy("etf_id", "etf_date", ascending=True)
    
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
#將Feature合併為Vector 並作標準化
assembler = VectorAssembler(
    inputCols=["EMA5", "EMA10", "EMA20", "BIAS", "K", "D", "DIF", "MACD", "RSI", "WR"],
    outputCol="features")
tot_dt_1 = assembler.transform(tot_dt)
#minmax_scaler = MinMaxScaler(inputCol="features", outputCol="stdFeatures")
#scaler_model = minmax_scaler.fit(tot_dt_1)
#std_scaler = StandardScaler(inputCol="features", outputCol="stdFeatures")
#scaler_model = std_scaler.fit(tot_dt_1)
#tot_dt_scale = scaler_model.transform(tot_dt_1)
tot_dt_scale=tot_dt_1.withColumn("stdFeatures", col("features")) #測試不作標準化

In [61]:
predict_start_date = "20180416"
#取出4/16~4/27 (共兩週資料作為測試集)
train_dt = tot_dt_scale.filter("etf_date < '" + predict_start_date + "' and MACD is not null") \
    .select("etf_id", "etf_date", "row_idx", "stdFeatures", "etf_close") \
    .orderBy("etf_id", "etf_date", ascending=True)
test_dt = tot_dt_scale.filter("etf_date >= '" + predict_start_date + "'") \
    .select("etf_id", "etf_date", "row_idx", "stdFeatures", "etf_close") \
    .orderBy("etf_id", "etf_date", ascending=True)
print('train count: ', str(train_dt.count()), ', test count: ', str(test_dt.count()))
train_dt.show(10)
test_dt.show(10)

train count:  6295 , test count:  50
+-------+--------+-------+--------------------+---------+
| etf_id|etf_date|row_idx|         stdFeatures|etf_close|
+-------+--------+-------+--------------------+---------+
|0050   |20130227|     35|[55.4823388864543...|     55.2|
|0050   |20130301|     36|[55.3882259243028...|     55.4|
|0050   |20130304|     37|[55.3921506162019...|    54.75|
|0050   |20130305|     38|[55.1781004108012...|     55.2|
|0050   |20130306|     39|[55.1854002738675...|    55.45|
|0050   |20130307|     40|[55.2736001825783...|     55.4|
|0050   |20130308|     41|[55.3157334550522...|     55.8|
|0050   |20130311|     42|[55.4771556367014...|     55.9|
|0050   |20130312|     43|[55.6181037578009...|    55.55|
|0050   |20130313|     44|[55.5954025052006...|    55.65|
+-------+--------+-------+--------------------+---------+
only showing top 10 rows

+-------+--------+-------+--------------------+---------+
| etf_id|etf_date|row_idx|         stdFeatures|etf_close|
+-------+

In [70]:
#test_dt.show(20, False)

In [62]:
train_dt.cache()
test_dt.cache()

DataFrame[etf_id: string, etf_date: string, row_idx: int, stdFeatures: vector, etf_close: double]

In [51]:
#取出etf的distinct id
etf_ids = []
for row in test_dt.select("etf_id").distinct().collect():
    etf_ids.append(row["etf_id"])
etf_ids

['0051   ', '0052   ', '0050   ', '0054   ', '0053   ']

In [53]:
#訓練Model及評估(RandomForestRegressor - etf_id wise) 
#-- accuracy:0.48, (RMSE) on test data = 0.938023 --> using stdScaler
#-- accuracy:0.48, (RMSE) on test data = 0.87064 --> no Scaler
#-- accuracy:0.56, (RMSE) on test data = 0.92546 --> use minmaxScaler
#-- accuracy:0.513812 , (RMSE) on test data = 0.597397 --> use minmaxScaler and no Scaler(full data)
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(featuresCol="stdFeatures",labelCol="etf_close")
#記下對應etf_id的所有model
model_map = {}
for etfid in etf_ids:
    train_data = train_dt.filter("etf_id='" + etfid + "'")
    rf_model = rf.fit(train_data)
    model_map.update({etfid: rf_model})

+-------+--------+-------+--------------------+---------+------------------+
| etf_id|etf_date|row_idx|         stdFeatures|etf_close|        prediction|
+-------+--------+-------+--------------------+---------+------------------+
|0051   |20180416|   1294|[32.6843289933820...|    33.18| 32.24945607851691|
|0051   |20180417|   1295|[32.7662193289213...|     33.0| 32.24945607851691|
|0051   |20180418|   1296|[32.9041462192809...|    32.75| 32.24945607851691|
|0051   |20180419|   1297|[32.9360974795206...|    33.15| 32.24945607851691|
|0051   |20180420|   1298|[32.8740649863470...|    33.02|32.226998487640216|
|0051   |20180423|   1299|[32.9660433242313...|     33.0| 32.24945607851691|
|0051   |20180424|   1300|[32.9840288828209...|    32.44| 32.24945607851691|
|0051   |20180425|   1301|[32.9893525885472...|    32.02| 32.24945607851691|
|0051   |20180426|   1302|[32.8062350590315...|    31.71|31.815579727771798|
|0051   |20180427|   1303|[32.5441567060210...|    32.11|31.449278786931778|

In [63]:
#find last records of all etf to be the base of next record
tetf_dt_prod = tetfp_dt2.filter("etf_date < '" + predict_start_date + "' and MACD is not null")
tetf_max_idx = tetf_dt_prod.groupBy("etf_id").max("row_idx")
tetf_max = tetf_max_idx.select(col("etf_id"), col("max(row_idx)").cast("Double").alias("row_idx")) \
    .join(tetf_dt_prod, ["etf_id", "row_idx"], "inner") \
    .select("etf_id", "etf_date", "row_idx", "etf_close", "close_price_raw", "dif_list") \
    .orderBy('etf_id')

tetf_max.show()

+-------+--------+-------+---------+--------------------+--------------------+
| etf_id|etf_date|row_idx|etf_close|     close_price_raw|            dif_list|
+-------+--------+-------+---------+--------------------+--------------------+
|0050   |20180413| 1293.0|     82.5|[54.4, 54.85, 54....|[0.35882671868262...|
|0051   |20180413| 1293.0|    32.93|[26.09, 26.12, 26...|[0.23810818793990...|
|0052   |20180413| 1293.0|    57.05|[32.72, 32.12, 32...|[0.30196772810150...|
|0053   |20180413| 1293.0|    36.46|[23.26, 23.11, 22...|[0.15597485321855...|
|0054   |20180413| 1293.0|    24.05|[19.4, 19.37, 19....|[0.09327655672724...|
+-------+--------+-------+---------+--------------------+--------------------+



In [75]:
#建立產生feature之方法
def create_feature(orig_dt):
    new_dt = orig_dt.withColumn("EMA5", calculate_ema(col("close_price_raw"), lit(5))) \
    .withColumn("EMA10", calculate_ema(col("close_price_raw"), lit(10))) \
    .withColumn("EMA20", calculate_ema(col("close_price_raw"), lit(20))) \
    .withColumn("BIAS", calculate_bias(col("close_price_raw"))) \
    .withColumn("KD", calculate_KD(col("close_price_raw"))) \
    .withColumn("K", col("KD")[0]).withColumn("D", col("KD")[1]) \
    .withColumn("MACD", calculate_MACD(col("dif_list"), col("DIF"))) \
    .withColumn("RSI", calculate_RSI(col("close_price_raw")))\
    .withColumn("WR", calculate_WR(col("close_price_raw")))
    return new_dt
#計算上或下的值(udf)
def judge_up_down_pred(curr_price, close_price_list):
    if len(close_price_list) == 0:
        return 0.0
    else:
        prev_price = close_price_list[-1]
        if curr_price == prev_price:
            return 0.0
        elif curr_price > prev_price:
            return 1.0
        else:
            return 2.0
judge_up_down_pred=udf(judge_up_down_pred, DoubleType())
#建立預測方法
def doPredict(test_dt, etf_ids):
    predict_res = None
    for etfid in etf_ids:
        test_data = test_dt.filter("etf_id='" + etfid + "'")
        predicts = rf_model.transform(test_data)
        if predit_res is None:
            predict_res = predicts
        else:
            predict_res = predict_res.unionAll(predicts)
    predict_res2 = predict_res.withColumn("pred_ud", judge_up_down_pred(col("prediction"), col("close_price_raw")))
    return predict_res2

In [67]:
schema = tetfp_dt2.select("etf_id", "etf_date", col("row_idx").cast("Double"), "close_price_raw", "DIF", "dif_list").schema
next_date_range = ["20180416", "20180417", "20180418","20180419", "20180420"]
idx_plus = 0.0
predict_res_final = None
for date in next_date_range:
    next_rows = []
    idx_plus = idx_plus +1
    for row in tetf_max.collect():
        close_price_raw = row["close_price_raw"]
        close_price_raw.append(row["etf_close"])
        dif_list = row["dif_list"]
        dif = calculate_DIF_native(close_price_raw)
        dif_list.append(dif)
        next_row = (row["etf_id"], date, row["row_idx"]+idx_plus, close_price_raw, dif ,dif_list)
        next_rows.append(next_row)
    orig_dt = spark.createDataFrame(next_rows, schema)
    new_dt = create_feature(orig_dt) #加入預測用的feature
    feature_dt = assembler.transform(new_dt).withColumn("stdFeatures", col("features")) #作出 FeatureVector
    #feature_dt.select("etf_id", "etf_date", "stdFeatures").show(10, False)
    #do predict using existed model
    pred_res = doPredict(feature_dt, etf_ids)
    #取出所有預測結果作合併，以進行後續成績計算
    merged_dt = predit_res.select("etf_id", "etf_date", "prediction", "pred_ud")
    if predict_res_final is None:
        predict_res_final = merged_dt
    else:
        predict_res_final = predict_res_final.unionAll(merged_dt)
    tetf_max = pred_res.select("etf_id", "etf_date", "row_idx", col("prediction").alias("etf_close"), "close_price_raw")
    
predict_res_final.show()

+-------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|etf_id |etf_date|stdFeatures                                                                                                                                                                                            |
+-------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0050   |20180416|[82.49563923275254,82.52734466707139,82.65634271500949,-0.45750000000001023,0.6196912499291656,0.4956054240339315,-0.15941862125276884,-0.1594186212527688,0.4954857100553611,0.3103448275862083]      |
|0051   |20180416|[32.76621932892135,32.61926812301739,32.42137407358534,0.5124999999999957,0.9336195439888764,0.85809845303

In [ ]:
#評估Model的RMES
from pyspark.ml.evaluation import RegressionEvaluator, MulticlassClassificationEvaluator
tetf_dt_eval = tetfp_dt2.filter("etf_date >= '" + predict_start_date + "'") \
    .select("etf_id", "etf_date", "etf_close", "close_price_raw")
eval_dt = predict_res_final.join(tetf_dt_eval, ["etf_id", "etf_date"], "inner") \
    .withColumn("act_ud", judge_up_down_pred(col("etf_close"), col("close_price_raw")))
eval_dt.show(20)

evaluator = RegressionEvaluator(
    labelCol="etf_close", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(eval_dt)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
evaluator = MulticlassClassificationEvaluator(
    labelCol="act_ud", predictionCol="pred_ud", metricName="accuracy")
accuracy = evaluator.evaluate(eval_dt)
print("accuracy = %g " % accuracy)

In [74]:
#ll = [46.92, 47.31, 47.0, 46.79, 46.49, 46.66, 47.0, 46.96, 47.0]
ll = [46.92, 47.31, 47.0, 46.79, 46.49, 46.66]
win_len=5
print(ll[0])
print(ll[0: -1])
print(list(map(lambda x,y : x - y, ll[1:], ll[0: -1])))
for x in ll[win_len:]:
    print(x)
ema = sum(ll[:win_len])/len(ll[:win_len])
print(ema)
for price in ll[win_len:]:
    ema = (ema*(win_len-1)+price*2)/(win_len+1)
tup1, tup2 = (1,2)
print(tup1, ' ', tup2)
tup = (3,4)
print(tup[0], ' ', tup[1])
list(range(0,2))
test_dic = {}
test_dic.update({"0051": assembler})
print(test_dic['0051'])

46.92
[46.92, 47.31, 47.0, 46.79, 46.49]
[0.39000000000000057, -0.3100000000000023, -0.21000000000000085, -0.29999999999999716, 0.1699999999999946]
46.66
46.902
1   2
3   4
VectorAssembler_49459e3f7e68da02f84b
